In [5]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
import threading
from jetbot import Camera, bgr8_to_jpeg
import cv2
import numpy as np
import tensorflow as tf
import time
import PIL
import io

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--modelfile", type=str, default="model1.tflite", help="tflite model file name for road following")
states = {}
stop_keys = [1,2,3,6] #our class keys
slow_keys = [4]
green_keys = [5]
left_keys = [7]
right_keys = [8]



camera = Camera.instance(width=224, height=224)

from jetbot import Robot
robot = Robot()

In [80]:
# Write some Text

font                   = cv2.FONT_HERSHEY_SIMPLEX
leftcorner = (10,10)
leftcorner2 = (10,30)
fontScale              = 0.5
fontColor              = (255,255,255)
lineType               = 2
prediction = ""
def display_prediction(camera_image):
    global prediction
    image = np.copy(camera_image)
    x = str(robot.left_motor.value)
    y = str(robot.right_motor.value)
    fulltext =  "left : " + x + " right : " + y

    cv2.putText(image,str(prediction), 
    leftcorner, 
    font, 
    fontScale,
    fontColor,
    lineType)
    cv2.putText(image,fulltext, 
    leftcorner2, 
    font, 
    fontScale,
    fontColor,
    lineType)    
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

image = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
target_link = traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_prediction)
#display(widgets.HBox([image]))
display(widgets.HBox([target_widget]))
#display(widgets.HBox([image,target_widget]))

In [ ]:
def changeCarState(info):
    x = info.split(",")

    for i in x:
        y = i.split(":")
        if int(y[0]) in states.keys():
            states[int(y[0])] = y[1]
        else:
            states.update({int(y[0]):y[1]})

def processStateChanged():
    if(any([i in states.keys() for i in stop_keys])):  #top priority
        robot.left_motor.value = 0.0
        robot.right_motor.value = 0.0
        print("Detected Stop")

In [81]:
# Creating function to interprete the image and output predictions
modelfilename = "3_refined_112112.tflite"
category = ["1straight", "2right", "3left"]
prediction = ""
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=modelfilename)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'conv2d_input', 'index': 15, 'shape': array([  1, 112, 112,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'dense_1/Softmax', 'index': 21, 'shape': array([1, 3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [82]:
def fixed_positioncheck(change):
    img = change['new']

    global states

    #overwrites by state manager

    if(len(states) > 0):
        processStateChanged()
        return
    
    # Line Tracing RGB
    if(modelfilename == "3_refined_168168.tflite"):
        img = PIL.Image.open(io.BytesIO(image.value)).resize((168,168))
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,168,168,3)
    elif(modelfilename == "3classes_refined_flipped.tflite"):
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,224,224,3)
    elif(modelfilename == "3_refined_crop164224.tflite"):
        #cropping 
        img = np.array(img).astype(np.float32)
        img_crop = img[60:,:]
        img = img_crop.reshape(1,164,224,3)
    elif(modelfilename == "3_refined_112112.tflite"):
        #test resize for in multiples. other wise use API
        input_size = 224
        output_size = 112
        bin_size = input_size // output_size
        img = img.reshape((output_size, bin_size, output_size, bin_size, 3)).max(3).max(1)
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,112,112,3)
    elif(modelfilename == "3_refined_cropresize112112.tflite"):
        #cropping
        start = time.process_time()
        #img = np.array(img).astype(np.float32)
        img_crop = img[60:,:]
        #resize
        img = PIL.Image.fromarray(img_crop).resize((112,112))
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,112,112,3)
        print('Preprocessing time : ', time.process_time() - start)
  
    
    #print(img.shape)
    start = time.process_time()
    #print(start)

    interpreter.set_tensor(input_details[0]['index'], img)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    index_predicted = np.argmax(output_data[0])
    global prediction
#     print (index_predicted)
    if category[index_predicted] != prediction:
        #print(category[index_predicted])
        prediction = category[index_predicted]
    if index_predicted == 0:
        #straight
        robot.left_motor.value = 0.27 #* speed_gain_slider.value
        robot.right_motor.value = 0.26 #* speed_gain_slider.value
        #print("To steer STRAIGHT")
    elif index_predicted == 1:
        #right
        #robot.right(0.2 * speed_gain_slider.value)
        robot.left_motor.value = 0.32 #* speed_gain_slider.value
        robot.right_motor.value = 0.12 #* speed_gain_slider.value
        #print("To steer RIGHT")
      
    elif index_predicted == 2:
        #left
        #robot.left(0.2 * speed_gain_slider.value)
        robot.left_motor.value = 0.12 #* speed_gain_slider.value
        robot.right_motor.value = 0.32 #* speed_gain_slider.value
        #print("To steer LEFT")
 
    print(time.process_time() - start)
    time.sleep(0.01)

In [11]:
def roadFollowing():
    #init once
    print("Start road following")
    for x in range(1):
        fixed_positioncheck({'new': camera.value})  # we call the function once to intialize

    robot.left_motor.value = 0.0
    robot.right_motor.value = 0.0

    time.sleep(1) 
    camera.observe(fixed_positioncheck, names='value')

In [ ]:
roadFollowing()

In [85]:
camera.unobserve_all()
robot.stop()

0.06720723400030693
0.06361994600001708
0.08763923400010754


In [83]:
for x in range(1):
    fixed_positioncheck({'new': camera.value})  # we call the function once to intialize
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

0.1049580710000555


In [84]:
roadFollowing()

Start road following
0.09303452999984074
0.0660484439999891
0.06257396000000881
0.06848464000040622
0.06636380100007955
0.06339473799971529
0.06896327999993446
0.06555364700034261
0.06371083199974237
0.06320765300006315
0.06591431999959241
0.06797912100000758
0.06295771299983244
0.06574635499964643
0.07010723499979576
0.07480285600013303
0.06548823399998582
0.06319291199997679
0.06480082500002027
0.06594895800026279
0.06417307600031563
0.06570056999998997
0.06691775800027244
0.06594187300015619
0.06467848900001627
0.06367677800017191
0.07019281600014438
0.06831291900016367
0.06501317600032053
0.0666527620001034
0.0704533379998793
0.06608354599984523
0.06697629500013136
0.07447041900013573
0.06765963500038197
0.06985973600012585
0.061795105999863154
0.06539530800000648
0.06815713100013454
0.06472114999996847
0.0676459450000948
0.07294671499994365
0.06651609499976985
0.06406890600010229
0.0667125540003326
0.06494832799990036
0.0670380669998849
0.06827125900008468
0.061241717000029894
0.0